# PROJECT 3_ Project web_Carlos Azagra

The goal of this project is practicing what I have learned in the APIs and Web Scraping chapter of this program. I will scrap two different websitdes related to football by using two different methods:

- **API - Football Data**: from which I will obtain different type of data such as participants, standings, matches, results or scorers from different competitions of different countries. By using this API I can only access to information about last 3 years.

- **Web Scrapping by reading HTML - BD Futbol**: from which I cannot obtain as much data as Football Data, but I have more access to historic data, which will allow me to get other type of data to get other analysis.

In [1]:
# Import the main libraries I will be using in both methods.

import pandas as pd
import os
import requests
import datetime as dt
from datetime import datetime
import math
from bs4 import BeautifulSoup
import lxml.html


## API_Football-Data.org

In [2]:
# Capsule the url from the API landing page

url = 'https://api.football-data.org/'

In [3]:
# Capsule the token as a dictionary to be used in requests

headers = { 'X-Auth-Token':  'ce63b7efde334f51be087b7fb4f5ccaa'}

In [4]:
# requests.get from the url with headers + capsule

resp = requests.get(url, headers=headers)

In [5]:
# Check the status of the requests

resp.status_code

200

In [6]:
# Check the url

resp.url

'https://www.football-data.org/'

### Query competitions - La Liga

In [7]:
# Create a class with functions related to COMPETITIONS

class FootballDataCompetition:

    # define the standard url and headers
    def __init__(self, url = 'https://api.football-data.org/v2/', headers = { 'X-Auth-Token':  'ce63b7efde334f51be087b7fb4f5ccaa'}):
        self.url = url
        self.headers = headers

    # get the id of the country of the competition I am interested in, by introducing the country name
    def areas (self, country = None):
        endpoint = '/areas'
        area1 = f'{self.url}{endpoint}'
        area2 = requests.get(area1, headers = headers).json()
        area3 = [(i['name'], i['id']) for i in area2['areas'] if i['name'] == country]
        return area3
    
    # get the id of the existing competitions in the desired country, by introducing the id of the country
    def areas2 (self, country = None):
        endpoint = '/competitions'
        area4 = f'{self.url}{endpoint}'
        params = {'areas': country}
        area5 = requests.get(area4, params, headers = headers).json()
        area6 = [(i['name'], i['id']) for i in area5['competitions']]
        return area6
              
    # get the teams that participate in a competition, by introducing the id of the competition. Can also introduce the season as a parameter season (standard season is current the one)
    def competition_teams (self, id_comp=None, season=None):
        endpoint = '/competitions/'
        endpoint1 = str(id_comp)
        endpoint2 = '/teams'
        url1 = f'{self.url}{endpoint}{endpoint1}{endpoint2}'
        params = {'season' : season}
        comp1 = requests.get(url1, params = params, headers = headers).json()
        teams_actual = [i['name'] for i in comp1['teams']]
        teams_table = pd.DataFrame(teams_actual)
        return teams_table 
    
    # get the standings of a competition, by introducing the id of the competition. Can also introduce the season as a parameter season (standard season is current the one)
    def competition_standings (self, id_comp=None, season=None):
        endpoint = '/competitions/'
        endpoint1 = str(id_comp)
        endpoint2 = '/standings'
        url2 = f'{self.url}{endpoint}{endpoint1}{endpoint2}'
        params = {'season' : season}
        comp2 = requests.get(url2, params = params, headers = headers).json()
        standings_table = pd.DataFrame(comp2['standings'][0]['table'])
        standings_table.set_index('position', inplace=True)
        standings_table['team'] = standings_table['team'].apply(lambda x: x['name'])
        return standings_table
        
    # get the top10 scorers of a competition, by introducing the id of the competition. Can also introduce the season as a parameter season (standard season is current the one)
    def competition_scorers (self, id_comp=None, season=None):
        endpoint = '/competitions/'
        endpoint1 = str(id_comp)
        endpoint2 = '/scorers'
        url3 = f'{self.url}{endpoint}{endpoint1}{endpoint2}'
        params = {'season' : season}
        comp3 = requests.get(url3, params = params, headers = headers).json()
        scorers_table = pd.DataFrame(comp3['scorers'])
        scorers_table['team'] =scorers_table['team'].apply(lambda x: x['name'])
        scorers_table['player'] = scorers_table['player'].apply(lambda x: x['name'])
        return scorers_table
    
    # get the next scheduled matchday of a competition, by introducing the id of the competition
    def competition_nextmatch (self, id_comp=None):
        endpoint = '/competitions/'
        endpoint1 = str(id_comp)
        endpoint2 = '/matches'
        url4 = f'{self.url}{endpoint}{endpoint1}{endpoint2}'
        comp4 = requests.get(url4, headers = headers).json()
        matches_sort = sorted([i['matchday'] for i in comp4['matches'] if i['status'] == 'SCHEDULED'])
        next_match = matches_sort[0]
        nxt = []
        for i in comp4['matches']:
            if i['matchday'] == next_match:
                nxt.extend([[i['homeTeam']['name'] + ' vs ' + i['awayTeam']['name'],
                i['utcDate'].split('T')[0],
                i['utcDate'].split('T')[1].replace('Z', '')]])
        next_match_table = pd.DataFrame(nxt, columns = ['match', 'date', 'time'])
        return next_match_table

In [8]:
# Create element la_liga

la_liga = FootballDataCompetition()

In [9]:
# Check id from Spain

la_liga.areas(country = 'Spain')

[('Spain', 2224)]

In [10]:
# Check competitions and its id in Spain

la_liga.areas2 (country = 2224)

[('Copa del Rey', 2079),
 ('Segunda División', 2077),
 ('Primera Division', 2014),
 ('Supercopa de España', 2078)]

In [11]:
# Call function competition_teams and export the returned DataFrame to csv
# Participant Teams in current season (2020)

la_liga.competition_teams(id_comp=2014).to_csv('API_output/1.laliga2020_teams.csv')

In [12]:
# Call function competition_nextmatch and export the returned DataFrame to csv
# Next matchday and its schedules in current season (2020)

la_liga.competition_nextmatch(id_comp=2014).to_csv('API_output/2.laliga2020_nextmatchday.csv')

In [13]:
# Call function competition_standings and export the returned DataFrame to csv
# Standings from last season (2019)

la_liga.competition_standings(id_comp=2014, season=2019).to_csv('API_output/3.laliga2019_standings.csv')

In [14]:
# Call function competition_scorers and export the returned DataFrame to csv
# Top10 scorers from two seasons ago (2018)

la_liga.competition_scorers(id_comp=2014, season=2018).to_csv('API_output/4.laliga2018_scorers.csv')

### Query teams - FC Barcelona

In [15]:
# Create a class with functions related to TEAMS

class FootballDataTeams():

    # define the standard url and headers
    def __init__(self, url = 'https://api.football-data.org/v2/', headers = { 'X-Auth-Token':  'ce63b7efde334f51be087b7fb4f5ccaa'}):
        self.url = url
        self.headers = headers

    # get the id of the country of the competition I am interested in, by introducing the country name
    def areas (self, country = None):
        endpoint = '/areas'
        area1 = f'{self.url}{endpoint}'
        area2 = requests.get(area1, headers = headers).json()
        area3 = [(i['name'], i['id']) for i in area2['areas'] if i['name'] == country]
        return area3
    
    # get the id of the existing competitions in the desired country, by introducing the id of the country
    def areas2 (self, country = None):
        endpoint = '/competitions'
        area4 = f'{self.url}{endpoint}'
        params = {'areas': country}
        area5 = requests.get(area4, params, headers = headers).json()
        area6 = [(i['name'], i['id']) for i in area5['competitions']]
        return area6
              
    # get the id and teams that participate in a competition, by introducing the id of the competition. Can also introduce the season as a parameter season (standard season is current the one)
    def competition_teams (self, id_comp=None, season=None):
        endpoint = '/competitions/'
        endpoint1 = str(id_comp)
        endpoint2 = '/teams'
        url1 = f'{self.url}{endpoint}{endpoint1}{endpoint2}'
        params = {'season' : season}
        team1 = requests.get(url1, params = params, headers = headers).json()
        teams_actual = [(i['id'], i['name']) for i in team1['teams']]
        return teams_actual 
    
    # get the info of the team I am interested, by introducing the id of the competition
    def teams_info (self, id_team=None, season=None):
        endpoint = '/teams/'
        endpoint1 = str(id_team)
        url2 = f'{self.url}{endpoint}{endpoint1}'
        params = {'season' : season}
        team2 = requests.get(url2, params = params, headers = headers).json()
        teaminfo_table = pd.DataFrame({'name':team2['name'],'address':team2['address'],'venue': team2['venue']},index=[1])
        return teaminfo_table
        
    # get the competitions in which the team participates, by introducing the id of the competition. Can also introduce the season as a parameter season (standard season is current the one)
    def teams_competitions (self, id_team=None, season=None):
        endpoint = '/teams/'
        endpoint1 = str(id_team)
        url3 = f'{self.url}{endpoint}{endpoint1}'
        params = {'season' : season}
        team3 = requests.get(url3, params = params, headers = headers).json()
        compinfo = [(i['name']) for i in team3['activeCompetitions']]
        compinfo_table = pd.DataFrame(compinfo, columns=['competition'])
        return compinfo_table
    
    # get the squad of the team, by introducing the id of the competition. Can also introduce the season as a parameter season (standard season is current the one)
    def teams_squad (self, id_team=None, season=None):
        endpoint = '/teams/'
        endpoint1 = str(id_team)
        url4 = f'{self.url}{endpoint}{endpoint1}'
        params = {'season' : season}
        team4 = requests.get(url4, params = params, headers = headers).json()
        squadinfo = [{'name':i['name'],'position':i['position'],'nationality':i['nationality'],'age':i['dateOfBirth']} for i in team4['squad'] if i['role']=='PLAYER']
        squadinfo_table = pd.DataFrame(squadinfo)
        squadinfo_table['age'] = squadinfo_table['age'].apply(lambda x: math.floor((dt.datetime.now() - dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ")).days/365))
        return squadinfo_table.sort_values(['position', 'name']).set_index('name').reset_index()
        

In [16]:
# Create element fcb

fcb = FootballDataTeams()

In [17]:
# Check id from Spain

fcb.areas(country = 'Spain')

[('Spain', 2224)]

In [18]:
# Check competitions and its id in Spain

la_liga.areas2 (country = 2224)

[('Copa del Rey', 2079),
 ('Segunda División', 2077),
 ('Primera Division', 2014),
 ('Supercopa de España', 2078)]

In [19]:
# Check teams and its id from Primera Division and return just tge FC Barcelona

for i in fcb.competition_teams(id_comp=2014):
    if 'FC Barcelona' in i:
        display(i)

(81, 'FC Barcelona')

In [20]:
# Call function teams_info and export the returned DataFrame to csv
# Basic club information

fcb.teams_info(id_team=81).to_csv('API_output/5.fcb_basicinfo.csv')

In [21]:
# Call function teams_competitions and export the returned DataFrame to csv
# Competitions in which the team competes this season 

fcb.teams_competitions(id_team=81).to_csv('API_output/6.fcb_competitions.csv')

In [24]:
# Call function teams_squad and export the returned DataFrame to csv
# Players from the squad

fcb.teams_squad(id_team=81).to_csv('API_output/7.fcb_squad.csv')

## Web Scrapping_Bdfutbol.com

In [25]:
# Capsule the url

url = 'https://www.bdfutbol.com/es/'

In [26]:
# requests.get from the url with headers + capsule

resp = requests.get(url)

In [27]:
# Check the status of the request

resp.status_code

200

In [28]:
# Obtain the HTML text

html = resp.text

In [29]:
# Read the HTML text by using lxml

root = lxml.html.fromstring(html)

In [30]:
# Create an lxml.html.HtmlElement element of Bdfutbol page
# Check the type of element created

type(root)

lxml.html.HtmlElement

In [31]:
# In the main page there are many competitions, so we need to look for the endpoint of laliga

laliga = root.xpath('/html/body/div[1]/div/div[9]/div[1]/div/div/div[2]/div[2]/span[2]/a/@href')

In [32]:
# Obtain HTML code from laliga

html2 = requests.get(url + ''.join(laliga)).text

In [33]:
# Create an lxml.html.HtmlElement element of Bdfutbol laliga page

root2 = lxml.html.fromstring(html2)

In [34]:
# In laliga page, we can find links to go to all history seasons
# Obtain the list of de historic of all the seasons in laliga. I will use them as endpoints

historico = root2.xpath('/html/body/div[1]/div/div[8]/div[2]/div[2]/div/div[@class="item_temporada"]/a/@href')

In [35]:
# Current season is not over - I do not consider it (exclude first element of the list)
# Obtain the list of de last 15 seasons when Messi was a player of FC Barcelona

seasons_w_messi = historico[1:16]

In [36]:
# Obtain the list of de previous 15 seasons when Messi was not a player of FC Barcelona

seasons_wo_messi = historico[16:31]

In [37]:
# Create empty lists to append each element per season with Messi: season year, played games, won games, draw games, lost games, scored goals and received goals

season = []
played_games = []
won_games = []
draw_games = []
lost_games = []
scored_goals = []
received_goals = []

# By using a for loop, I get all the different links from which I will obtain the standings and all the info needed:

for i in seasons_w_messi:
    html3 = requests.get(url + '/t/' + i).text
    root3 = lxml.html.fromstring(html3)
    season.append(''.join(root3.xpath('//body/div[1]/div/div[11]/h1/span[3]/text()')))
    played_games.append(''.join(root3.xpath('//tr[@ideq="1"]/td[6]/text()')))
    won_games.append(''.join(root3.xpath('//tr[@ideq="1"]/td[7][@class="resp_off"]/text()')))
    draw_games.append(''.join(root3.xpath('//tr[@ideq="1"]/td[8][@class="resp_off"]/text()')))
    lost_games.append(''.join(root3.xpath('//tr[@ideq="1"]/td[9][@class="resp_off"]/text()')))
    scored_goals.append(''.join(root3.xpath('//tr[@ideq="1"]/td[10][@class="resp_off"]/text()')))
    received_goals.append(''.join(root3.xpath('//tr[@ideq="1"]/td[11][@class="resp_off"]/text()')))

In [39]:
# Define a dictionary that will be used to creat a DataFrame with all the stats per season

games = { 'played_games':played_games, 'won_games':won_games, 'draw_games':draw_games, 'lost_games':lost_games, 'scored_goals':scored_goals, 'received_goals':received_goals}

# Creation of the DataFrame

df_w_messi = pd.DataFrame(games, columns=['played_games','won_games', 'draw_games', 'lost_games', 'scored_goals', 'received_goals'], index=season)

# Casting from str to int

df_w_messi = df_w_messi.applymap(lambda x: int(x))

# Creation of a csv file with the DataFrame

df_w_messi.to_csv('WS_output/1.standings_with_messi.csv')

In [40]:
# Define a dictionary that will be used to creat a DataFrame with all the stats of all seasons together

total_games = { 'total_played_games':df_w_messi['played_games'].sum(), 'total_won_games':df_w_messi['won_games'].sum(), 'total_draw_games':df_w_messi['draw_games'].sum(), 'total_lost_games':df_w_messi['lost_games'].sum(), 'total_scored_goals':df_w_messi['scored_goals'].sum(), 'total_received_goals':df_w_messi['received_goals'].sum()}

# Creation of the DataFrame

df_w_messi2 = pd.DataFrame(total_games, columns=['total_played_games','total_won_games', 'total_draw_games', 'total_lost_games', 'total_scored_goals', 'total_received_goals'], index=['fcb_w_messi'])

In [42]:
# Create empty lists to append each element per season without Messi: season year, played games, won games, draw games, lost games, scored goals and received goals.

season1 = []
played_games1 = []
won_games1 = []
draw_games1 = []
lost_games1 = []
scored_goals1 = []
received_goals1 = []

# By using a for loop, I get all the different links from which I will obtain the standings and all the info needed

for i in seasons_wo_messi:
    html4 = requests.get(url + '/t/' + i).text
    root4 = lxml.html.fromstring(html4)
    season1.append(''.join(root4.xpath('//body/div[1]/div/div[11]/h1/span[3]/text()')))
    played_games1.append(''.join(root4.xpath('//tr[@ideq="1"]/td[6]/text()')))
    won_games1.append(''.join(root4.xpath('//tr[@ideq="1"]/td[7][@class="resp_off"]/text()')))
    draw_games1.append(''.join(root4.xpath('//tr[@ideq="1"]/td[8][@class="resp_off"]/text()')))
    lost_games1.append(''.join(root4.xpath('//tr[@ideq="1"]/td[9][@class="resp_off"]/text()')))
    scored_goals1.append(''.join(root4.xpath('//tr[@ideq="1"]/td[10][@class="resp_off"]/text()')))
    received_goals1.append(''.join(root4.xpath('//tr[@ideq="1"]/td[11][@class="resp_off"]/text()')))

In [43]:
# Define a dictionary that will be used to creat a DataFrame with all the stats per season

games1 = { 'played_games1':played_games1, 'won_games1':won_games1, 'draw_games1':draw_games1, 'lost_games1':lost_games1, 'scored_goals1':scored_goals1, 'received_goals1':received_goals1}

# Creation of the DataFrame

df_wo_messi = pd.DataFrame(games1, columns=['played_games1','won_games1', 'draw_games1', 'lost_games1', 'scored_goals1', 'received_goals1'], index=season1)

# Casting from str to int

df_wo_messi = df_wo_messi.applymap(lambda x: int(x))

# Creation of a csv file with the DataFrame

df_wo_messi.to_csv('WS_output/2.standings_without_messi.csv')

In [44]:
# Define a dictionary that will be used to creat a DataFrame with all the stats of all seasons together

total_games1 = { 'total_played_games':df_wo_messi['played_games1'].sum(), 'total_won_games':df_wo_messi['won_games1'].sum(), 'total_draw_games':df_wo_messi['draw_games1'].sum(), 'total_lost_games':df_wo_messi['lost_games1'].sum(), 'total_scored_goals':df_wo_messi['scored_goals1'].sum(), 'total_received_goals':df_wo_messi['received_goals1'].sum()}

# Creation of the DataFrame

df_wo_messi2 = pd.DataFrame(total_games1, columns=['total_played_games','total_won_games', 'total_draw_games', 'total_lost_games', 'total_scored_goals', 'total_received_goals'], index=['fcb_wo_messi'])

In [45]:
# Concatenate both DataFrames to get all figures from FC Barcelona with and without Messi

comparision = pd.concat([df_w_messi2, df_wo_messi2])

# Creation of a csv file with the DataFrame

comparision.to_csv('WS_output/3.total_standings.csv')

In [46]:
# Obtain the % between the figures with Messi and without Messi

comparision2 = [comparision.iloc[0,:] / comparision.iloc[1,:]]

In [47]:
# Creation of the DataFrame

comparision3 = pd.DataFrame(comparision2, columns = ['total_played_games', 'total_won_games', 'total_draw_games', 'total_lost_games', 'total_scored_goals', 'total_received_goals'], index = ['messi_vs_nomessi'])

# Creation of a csv file with the DataFrame

comparision3.to_csv('WS_output/4.messi_vs_no_messi.csv')

In [48]:
comparision3

,total_played_games,total_won_games,total_draw_games,total_lost_games,total_scored_goals,total_received_goals
messi_vs_nomessi,0.986159,1.240854,0.776923,0.516667,1.279514,0.764241


From the stats obtained, despite having played 8 matches less (-1,4%), we can see that FC Barcelona has the following figures with Messi as a player:

- Total games won: 24% more with Messi
- Total games draw: 22% less with Messi
- Total games lost: 49% less with Messi
- Total goals scored: 28% more with Messi
- Total goals received: 24% less with Messi

We can see that all single stats have improved with Messi in the team.